<a href="https://colab.research.google.com/github/orenbara/braude_cloudcomp/blob/master/hw2/web_app/OnShapeLogVisualizationTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
!pip install firebase

### Connection to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Imports

In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files, drive
import matplotlib.colors as mcolors
from firebase import firebase
import numpy as np
from IPython.display import YouTubeVideo

### Load JSON from Google Drive to Firebase DB

In [ ]:
FBconn = firebase.FirebaseApplication('https://tirgul-db-default-rtdb.firebaseio.com/',None)

create_db = "False" # @param {type:"string"}
comment = "!!!!!!Change create_db to True only if you have access to Anna's drive!!!!!!" # @param {type:"string"}

if create_db:
  # Create and upload the empty JSON data to Firebase
  FBconn.delete('/', None)
  empty_data = {}
  file_path_empty_json = '/content/drive/My Drive/Colab Notebooks/CloudHW2/Empty.json'
  with open(file_path_empty_json, 'w') as f:
      json.dump(empty_data, f)
  with open(file_path_empty_json) as f:
      data = json.load(f)
  result_put = FBconn.put('/', 'empty_data', data)

  # Load JSON data
  file_path = '/content/drive/My Drive/Colab Notebooks/CloudHW2/OnShapeLog.json'
  with open(file_path) as f:
      data = json.load(f)
  result_put = FBconn.put('/', 'OnShapeJsonData', data)
else:
  print("You didn't created the data base")


# Convert JSON data to a Data Frame
result_get = FBconn.get('/OnShapeJsonData/',None)
df = pd.DataFrame(result_get)

### Additional feature: Youtube website as a SaaS (Software as a Service) component

In [ ]:
def display_video():
    video = YouTubeVideo('VDK1A5KIf7g', width=600, height=300)
    display(video)

### Dashboard

In [ ]:
# Helper function to create a modern navigation bar
def create_navbar():
    navbar = widgets.ToggleButtons(
        options=['Dashboard', 'Students Activity', 'Project Activity', 'FAQ'],
        button_style='',
        layout=widgets.Layout(width='100%', justify_content='center', margin='0 0 20px 0')  # Added margin for spacing
    )
    navbar.style.button_width = 'auto'
    navbar.style.button_color = '#D3E6F3'
    navbar.style.font_weight = 'bold'
    return navbar

# Helper function to switch between pages
def on_nav_change(change):
    if change['new'] == 'Dashboard':
        display_main_dashboard()
    elif change['new'] == 'Students Activity':
        display_students_activity_dashboard()
    elif change['new'] == 'Project Activity':
        display_project_activity_dashboard()
    elif change['new'] == 'FAQ':
        display_faq_page()


# Main Dashboard
def plot_most_active_users():
    plt.figure(figsize=(5, 5))
    most_active_users = df['User'].value_counts().head(10)

    # Calculate the percentage for each user
    total = most_active_users.sum()
    percentages = most_active_users / total * 100

    # Generate colors using a blue color map
    colors = plt.cm.Blues(np.linspace(0.4, 0.8, len(most_active_users)))

    # Create the pie chart
    plt.pie(percentages, labels=percentages.index, autopct='%1.1f%%', startangle=90, colors=colors)

    # Add a circle at the center to create a donut chart effect
    center_circle = plt.Circle((0,0), 0.70, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(center_circle)

    plt.title('Most Active Users', fontsize=15, fontweight='bold', color='#061B35')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

    # Add a legend
    plt.legend(percentages.index, title="Users", bbox_to_anchor=(1, 0, 0.5, 1))

    plt.show()

def display_main_dashboard():
    clear_output(wait=True)
    display(navbar)
    latest_updates = df.sort_values(by='Time', ascending=False).head(10)

    # Create a layout with two columns
    layout = widgets.Layout(display='flex', flex_flow='row', justify_content='space-between')
    left_column = widgets.Box(layout=widgets.Layout(width='60%'))
    right_column = widgets.VBox(layout=widgets.Layout(width='35%'))  # Changed to VBox for vertical stacking

    # Common title HTML
    title_html = """
    <style>
        @import url('https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css');
    </style>
    <div class="text-center mb-4">
        <h1 class="text-4xl font-bold">Dashboard</h1>
        <hr class="mt-2">
    </div>
    """

    # Table HTML (left column)
    table_html = f"""
    <style>
        table {{
            border-collapse: collapse;
            font-size: 16px;
            text-align: right;
            width: 100%;
        }}

        table th, table td {{
            padding: 8px;
            border: 1px solid #2196F3;
            text-align: right;
        }}

        table th {{
            background-color: #E3F2FD;
            text-align: center;
        }}

        table tr:nth-child(even) {{
            background-color: #E3F2FD;
        }}

        table tr:nth-child(odd) {{
            background-color: white;
        }}

        table tr:hover {{
            background-color: #BBDEFB;
        }}
                /* Adjust column widths */
        th:nth-child(1), td:nth-child(1) {{
            width: 35%;
        }}

        th:nth-child(2), td:nth-child(2) {{
            width: 20%;
            justify-content: right;
        }}

        th:nth-child(3), td:nth-child(3) {{
            width: 15%;
            justify-content: right;
        }}

        th:nth-child(4), td:nth-child(4) {{
            width: 20%;
            justify-content: right;
        }}
        th:nth-child(), td:nth-child() {{
            width: 10%;
            justify-content: right;
        }}
    </style>
    <h3 class="text-2xl mb-4 font-bold text-center">Latest Updates</h3>
    {latest_updates.to_html(classes='table', index=False)}
    """
    left_column.children = [widgets.HTML(table_html)]

    # Plot and YouTube video (right column)
    right_column_content = widgets.VBox()  # Create a VBox to stack plot and video
    plot_output = widgets.Output()
    video_output = widgets.Output()

    with plot_output:
        plot_most_active_users()

    with video_output:
        display_video()

    right_column_content.children = [plot_output, video_output]
    right_column.children = [right_column_content]

    # Display the layout
    display(widgets.HTML(title_html))
    display(widgets.HBox([left_column, right_column], layout=layout))



# Students Activity Dashboard
def display_students_activity_dashboard():
    clear_output(wait=True)
    display(navbar)
    display(HTML("""
    <style>
        @import url('https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css');
    </style>
    <div class="text-center">
        <h1 class="text-4xl font-bold mb-4">Students Activity</h1>
    </div>
    """))
    display(dashboard)

# Project Activity Dashboard
def display_project_activity_dashboard():
    clear_output(wait=True)
    display(navbar)
    display(HTML("""
    <style>
        @import url('https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css');
    </style>
    <div class="text-center">
        <h1 class="text-4xl font-bold mb-4">Project Activity</h1>
    </div>
    """))
    display(project_dashboard)

# FAQ Page
def display_faq_page():
    clear_output(wait=True)
    display(navbar)
    display(HTML("""
    <style>
        @import url('https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css');
    </style>
    <div class="text-center">
        <h1 class="text-4xl font-bold mb-4">FAQ</h1>
        <p>Here you can find the frequently asked questions...</p>
    </div>
    """))

# Create and display the main dashboard with navigation bar
navbar = create_navbar()
navbar.observe(on_nav_change, names='value')

display_main_dashboard()


ToggleButtons(layout=Layout(justify_content='center', margin='0 0 20px 0', width='100%'), options=('Dashboard'…

Accordion(children=(HTML(value='This program is a visualization tool designed to help project managers analyze…

Accordion(children=(HTML(value="Place a JSON file called 'OnShapeLog' in the One Drive folder where the Google…

Accordion(children=(HTML(value='1. Project Distribution: A pie chart showing the distribution of all projects.…

Accordion(children=(HTML(value='A Gantt chart is a type of bar chart that represents a project schedule, showi…

Accordion(children=(HTML(value='Select the parameters you want to visualize using the multi-select widget. The…

Accordion(children=(HTML(value='Yes, you can filter the visualizations by user using the filter options provid…

Accordion(children=(HTML(value='Check the FAQ section for common issues and troubleshooting steps. If the prob…

Accordion(children=(HTML(value='Unfortunately, this option is not yet available. We hope to fix this in a futu…

Accordion(children=(HTML(value='Refer to the explanation provided above each chart.'),), _titles={'0': 'How ca…

### Students Activity Dashboard

In [ ]:
# Function to plot the most common actions by users
def plot_most_common_actions(user=None):
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This chart displays the most common actions taken by students.</p>
        <p>The frequency of each action is shown on the y-axis, and the actions are listed with numbers on the x-axis.</p>
        <p>The legend on the right explains the different actions performed.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(10, 6))
    if user:
        filtered_df = df[df['User'] == user]
    else:
        filtered_df = df
    most_common_actions = filtered_df['Description'].value_counts().head(10)
    most_common_actions.plot(kind='bar')
    x_pos = np.arange(len(most_common_actions))
    plt.bar(x_pos, most_common_actions.values, align='center', alpha=0.7, color=plt.cm.Blues(np.linspace(0.4, 0.8, len(most_common_actions))))
    plt.xticks(x_pos, [str(i+1) for i in x_pos], fontsize=10, rotation=45)
    plt.ylabel('Frequency', fontsize=12)
    plt.xlabel('Description', fontsize=12)
    plt.title('Most Common Actions', fontsize=15, fontweight='bold', color='#061B35')

    # Add legend
    legend_labels = [f'{i+1} - {name}' for i, name in enumerate(most_common_actions.index)]
    legend_text = "\n".join(legend_labels)
    legend_title = "Actions description\n"
    plt.gcf().text(1.05, 0.5, legend_title + legend_text, fontsize=10, verticalalignment='center', bbox=dict(facecolor='white', alpha=0.5))

    plt.show()

# Function to plot the distribution of actions over time
def plot_time_distribution(user=None):
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This chart displays the distribution of actions over time.</p>
        <p>The x-axis represents the hours of the day, while the y-axis shows the frequency of actions taken during each hour.</p>
        <p>It helps to understand at which times of the day the actions are most commonly performed by students.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(12, 6))
    if user:
        filtered_df = df[df['User'] == user]
    else:
        filtered_df = df

    # Extract hour and create a distribution
    time_distribution = filtered_df['Time'].apply(lambda x: int(x.split(' ')[1].split(':')[0])).value_counts().sort_index()

    # Create a full range of hours and fill with zeros where no data
    full_hours = pd.Series(0, index=range(24))
    time_distribution = full_hours.add(time_distribution, fill_value=0)

    time_distribution.plot(kind='line', marker='o')
    plt.title('Time Distribution of Actions')
    plt.xlabel('Hour of Day')
    plt.ylabel('Frequency')
    plt.xticks(range(0, 24))
    plt.xlim(0, 23)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Widgets for Students Activity Dashboard
user_options = ['All Users'] + sorted(df['User'].dropna().unique().tolist())
user_selector = widgets.Dropdown(options=user_options, description='Select User:', layout=widgets.Layout(width='50%'),style={'description_width': 'initial'})

parameter_options = ['Most Common Actions', 'Time Distribution']
parameter_selector = widgets.SelectMultiple(options=parameter_options, description='Select Parameters:', rows=3, layout=widgets.Layout(width='50%'),style={'description_width': 'initial'})


def update_dashboard(change):
    user = user_selector.value if user_selector.value != 'All Users' else None
    parameters = parameter_selector.value
    clear_output(wait=True)
    display(navbar)
    display(dashboard)

    if 'Most Common Actions' in parameters:
        plot_most_common_actions(user)
    if 'Time Distribution' in parameters:
        plot_time_distribution(user)

user_selector.observe(update_dashboard, names='value')
parameter_selector.observe(update_dashboard, names='value')

dashboard = widgets.VBox([
    widgets.HTML("<hr>"),
    widgets.HBox([user_selector, parameter_selector]),
    widgets.HTML("<hr>")
])


### Project Activity

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Assuming df is your DataFrame containing project data

# Function to plot the distribution of projects using a pie chart
def plot_project_distribution():
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This pie chart displays the distribution of projects based on the number of documents associated with each project.</p>
        <p>Each slice represents a project, and the percentage next to each slice indicates its share of the total documents.</p>
        <p>The chart is sorted to show the all projects by document count.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(10, 6))
    project_distribution = df['Document'].value_counts()
    project_distribution.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title('Project Distribution')
    plt.ylabel('')  # Hide the y-label for better visualization
    plt.show()

# Function to plot the time activity distribution of projects using a bar chart
def plot_project_time_activity():
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This bar chart displays the distribution of project activities over different hours of the day.</p>
        <p>Each bar represents the frequency of activities performed during a specific hour.</p>
        <p>The x-axis shows the hours of the day, and the y-axis represents the frequency of activities.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(10, 6))
    time_activity = df['Time'].apply(lambda x: x.split(' ')[1].split(':')[0]).value_counts().sort_index()
    time_activity.plot(kind='line')
    plt.title('Project Time Activity')
    plt.xlabel('Hour of Day')
    plt.ylabel('Frequency')
    plt.show()

# Function to plot specific activity for a given project using a scatter plot
def plot_project_specific_activity(project):
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This scatter plot displays the frequency of specific activities for a given project.</p>
        <p>Each point represents an activity, and its position along the x-axis shows the activity's description.</p>
        <p>The y-axis represents the frequency of each activity.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(10, 6))
    filtered_df = df[df['Document'] == project]
    specific_activity = filtered_df['Description'].value_counts().head(10).reset_index()
    specific_activity.columns = ['Description', 'Count']
    x_positions = range(1, len(specific_activity) + 1)
    plt.scatter(x_positions, specific_activity['Count'])

    # Set xticks to show activity indices
    plt.xticks(x_positions, x_positions)
    plt.title(f'Activity for Project: {project}')
    plt.xlabel('Activity Index')
    plt.ylabel('Frequency')

    # Add legend
    legend_labels = [f'{i+1} - {desc}' for i, desc in enumerate(specific_activity['Description'])]
    legend_title = "Activity Description\n"
    plt.gcf().text(1.05, 0.5, legend_title + "\n".join(legend_labels), fontsize=10, verticalalignment='center', bbox=dict(facecolor='white', alpha=0.5))

    plt.show()

# Function to plot user activity for a given project using a horizontal bar chart
def plot_project_user_activity(project):
    # Explanation HTML
    explanation_html = """
    <div style="background-color: white; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
        <h4 style="margin-top: 0;">Plot Explanation</h4>
        <p>This horizontal bar chart displays the frequency of user activity for a given project.</p>
        <p>Each bar represents a user, and the length of the bar corresponds to the frequency of their activity.</p>
        <p>The y-axis shows the list of users, and the x-axis represents the frequency of activities.</p>
    </div>
    """
    display(HTML(explanation_html))

    # Plot
    plt.figure(figsize=(10, 6))
    filtered_df = df[df['Document'] == project]
    user_activity = filtered_df['User'].value_counts()
    user_activity.plot(kind='barh')
    plt.title(f'User Activity for Project: {project}')
    plt.xlabel('Frequency')
    plt.ylabel('User')
    plt.show()

# Function to get project options from the DataFrame
def get_project_options():
    document_column = 'Document'
    if document_column in df.columns:
        # Drop NaN values and convert all entries to strings
        document_values = df[document_column].dropna().astype(str).unique()
        # Filter out any remaining empty strings or strings that were originally floats
        document_values = [p for p in document_values if p.strip() != '']
        # Sort the list of document values
        project_options = ['All Projects'] + sorted(document_values)
    else:
        raise KeyError(f"Column '{document_column}' not found in DataFrame")
    return project_options

# Function to update parameter options based on project selection
def update_parameter_options(change):
    if project_selector.value == 'All Projects':
        project_parameter_selector.options = all_projects_parameters
    else:
        project_parameter_selector.options = specific_project_parameters

# Function to update the project dashboard based on project and parameter selection
def update_project_dashboard(change):
    project = project_selector.value
    parameters = project_parameter_selector.value
    clear_output(wait=True)
    display(navbar)
    display(project_dashboard)

    if project == 'All Projects':
        if 'Project Distribution' in parameters:
            plot_project_distribution()
        if 'Project Time Activity' in parameters:
            plot_project_time_activity()
    elif project:
        if 'Project Specific Activity' in parameters:
            plot_project_specific_activity(project)
        if 'Project User Activity' in parameters:
            plot_project_user_activity(project)

# Get project options from DataFrame
project_options = get_project_options()

# Initialize project selector with default value 'All Projects'
project_selector = widgets.Dropdown(
    options=project_options,
    value='All Projects',
    description='Select Project:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# Define parameters for project plots selection
all_projects_parameters = ['Project Distribution', 'Project Time Activity']
specific_project_parameters = ['Project Specific Activity', 'Project User Activity']

# Initialize parameter selector for project dashboard with default value selections for 'All Projects'
project_parameter_selector = widgets.SelectMultiple(
    description='Select Parameters:',
    rows=3,
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
project_parameter_selector.options = all_projects_parameters

# Observe changes in project selector and update parameter options
project_selector.observe(update_parameter_options, names='value')
project_parameter_selector.observe(update_project_dashboard, names='value')

# Observe changes in project selector and parameter selector, and update project dashboard
project_selector.observe(update_project_dashboard, names='value')
project_parameter_selector.observe(update_project_dashboard, names='value')

# Define project dashboard layout
project_dashboard = widgets.VBox([
    widgets.HTML("<hr>"),
    widgets.HBox([project_selector, project_parameter_selector]),
    widgets.HTML("<hr>")
])

# Display initial project dashboard
display(project_dashboard)


### FAQ Page

In [ ]:
# FAQ Page
# Define questions and answers
faq_data = {
    "What is this program?": "This program is a visualization tool designed to help project managers analyze the actions of students on the OnShape platform. It provides various charts and plots to understand project distributions, user activities, and time-based actions.",
    "How do I upload the log file?": "Place a JSON file called 'OnShapeLog' in the One Drive folder where the Google Cloud file with the program project is located.",
    "What kind of visualizations are available?": "1. Project Distribution: A pie chart showing the distribution of all projects.<br>2. Time Activity Distribution: A line chart showing the frequency of actions taken during each hour of the day.<br>3. Specific Activity for a Project: A scatter plot displaying the frequency of specific activities within a given project.<br>4. User Activity for a Project: A horizontal bar chart showing user activity within a specific project.",
    "How do I select a project for detailed analysis?": "A Gantt chart is a type of bar chart that represents a project schedule, showing the start and finish dates of elements of a project.",
    "How do I choose which parameters to visualize?": "Select the parameters you want to visualize using the multi-select widget. The available parameters will update based on the selected project.",
    "Can I filter the visualizations by user?": "Yes, you can filter the visualizations by user using the filter options provided in the program.",
    "What if I encounter an error?": "Check the FAQ section for common issues and troubleshooting steps. If the problem persists, contact the support team: *9099",
    "How can I export the visualizations?": "Unfortunately, this option is not yet available. We hope to fix this in a future update.",
    "How can I get more details about a specific chart?": "Refer to the explanation provided above each chart."
}

# Function to create a widget for each question
def create_faq_widget(question, answer):
    accordion = widgets.Accordion(children=[widgets.HTML(value=answer)])
    accordion.set_title(0, question)
    return accordion

# Create FAQ widgets
faq_widgets = [create_faq_widget(question, answer) for question, answer in faq_data.items()]

def display_faq_page():
    clear_output(wait=True)
    display(navbar)
    display(HTML("<h1 style='text-align: center;'>FAQ for Project Managers</h1>"))
    for widget in faq_widgets:
        display(widget)
        widget.selected_index = None  # Ensure all items start closed

# Create and display the main dashboard with navigation bar
navbar = create_navbar()
navbar.observe(on_nav_change, names='value')

display_main_dashboard()

ToggleButtons(layout=Layout(justify_content='center', margin='0 0 20px 0', width='100%'), options=('Dashboard'…

HTML(value='\n    <style>\n        @import url(\'https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tai…